# MNIST Para Principiantes

En esta libreta se realizara el tutorial de MNIST del TensorFlow. Puedes verlo dando clic [aqui](URL "https://www.tensorflow.org/versions/master/tutorials/mnist/beginners/index.html").

El objetivo de realizar esta libreta es de ver la capacidad del TensorFlow.

MNIST es un sistema simple que trabaja con un conjunto de datos que consisten en imagenes escritas a mano como estos:

![Image](https://www.tensorflow.org/versions/master/images/MNIST.png)

Tambien incluye etiquetas por cada imagen, diciendo que digito es que. por ejemplo las etiquetas de las imagenes de arriba serian 5, 0, 4 y 1.

En este Tutorial vamos a entrenar un modelo para que vea unas imagenes y predecira que digito son. 


## The MNIST Data

Los datos Para el MNIST se descaragan con el achivo **input_data.py** que se ejecuta de la siguiente forma.

In [2]:
import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


La informacion descargada es separada en 3 partes, 55,000 datos para entrenamiento, 10,000 datos para prueba y 5,000 datos para validacion. esta separacion es muy importante: es escencial en machine learning separar la informacion con la que no aprendemos para verificar que de verdad aprendimos.

Como se menciono anteriormente, cada dato en el MNIST tiene dos partes: una imagen de algun digito escrito a mano y una etiqueta de a que valor corresponde. Llamaremos a estas imagenes "xs" y a las etiquetas "ys". tanto como el set de entrenamiento como el de prueva contienen xs y ys, por ejemplo las imagenes de entrenamiento son **mnist.train.images** y las etiquetas de entrenamiento son **mnist.train.labels**.

Cada imagen es de 28x28 pixeles, donde esto se puede interpretar como una vector de numeros.

![](https://www.tensorflow.org/versions/master/images/MNIST-Matrix.png)

El resultado que **mnist.train.images** es un tensor(un arreglo n dimensional) con una forma de **[55000, 784]**. la primer dimension es para indexar las imagenes y la segunda para los pixeles de cada imagen. cada entrada en el tensor es la intensidad del pixel con valor entre 0 y 1.

![](https://www.tensorflow.org/versions/master/images/mnist-train-xs.png)

Las etiquetas correspondientes en el MNIST son numeros entre 0 y 9, diciendo que digito es dado a cada imagen. Para el proposito de este tutorial, vamos a querer que nuestras etiquetas como un "one-hot vectors". esto es un vector que es 0 en casi todas las dimensiones y 1 en solo una dimension. en este caso el n-esimo digito va a ser representado como un vector en donde es uno en la n-esima dimension. por ejemplo 3 seria **[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]**. Por lo tanto, **mnist.train.labels** ss un arreglo de flotantes de la forma **[55000, 10]**.

![](https://www.tensorflow.org/versions/master/images/mnist-train-ys.png)

Ahora ya estamos listos para hacer nuestro modelo!


# Implementando Regresion Softmax

Se importa el TensorFlow:

In [3]:
import tensorflow as tf

Creamos un placeholder:

In [4]:
x = tf.placeholder(tf.float32, [None, 784])

x es donde se guardara la instruccion para el tensorflow por eso es un placeholder.

Tabien necesitamos pesos y bias para el modelo.

In [5]:
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

A estas variables se les da un valor inicial de 0 en todo sus puntos.

Ahora podemos implementar nuestro modelo utilizando solamente una linea!

In [6]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

Primero multiplicamos x por W, despues se suma b al resultado y despues se aplica la funcion softmax de tensorflow

# Entrenamiento

para entrenar nuestro modelo, necesitamos definir que significa que el modelo sea bueno.

para esto vamos a ulizar una distribucion conocida como cross-entropy que mide que tan ineficiente son nuestras predicciones para decir la verdad.

para implementar el cross-entropy necesitamos añadir un nuevo placeholder para poner la respuesta correcta.

In [7]:
y_ = tf.placeholder(tf.float32, [None, 10])

Despues implementamos el cross-entropy:

In [8]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

Primero tf.log calcula el logaritmo de cada elemento de y, despues multiplicamos cada elemento de y_ con su elemento correspondiente en tf.log(y). por ultimo tf.reduce_sum suma todos los elementos del tensor.

ahora que ya se sabe que es lo que queremos que realize nuestro modelo es muy facil entrenar al tensorflow.
vamos a utilizar el algoritmo de backpropagation para determinar eficientemente como nuestras variables afectan el costo que pides minimizar.


In [9]:
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

En caso de que le pidamos al TensorFlow que minimize el cross_entropy utilizando el algoritmo de decenso de gradiente con un aprendizaje del 0.01.

ahora tenemos nuestro modelo listo para entrenar. nomas falta agregar una operacion para inicializar todas las variables que hemos creado.

In [10]:
init = tf.initialize_all_variables()

Ahora iniciamos nuestro modelo en una **Session** y ejecutamos la operacion que inicializa las variables.

In [11]:
sess = tf.Session()
sess.run(init)

Vamos a entenar 1000 veces.

In [12]:
for i in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

En cada iteracion se obtiene un batch de 100 datos al azar para el conjunto de entrenamiento. Ejecutamos **train_step** para alimentar los batches reemplazando los placeholders.

# Evaluando el Modelo

Primero vamos a predecir donde se predijo una etiqueta correcta. **tf.argmax** es una funcion muy util que te da el indice de la entrada mas alta en el tensor asi como su posicion. podemos utilizar  **tf.equal** para checar si nuestras predicciones son correctas.

In [13]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

Esto nos da una lista de booleanos. para determinar que partes estan correctas, llamamos a la media de estos puntos.

In [14]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

Por ultimo, pedimos la exactitud de los datos de prueba.

In [15]:
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.905


Esto es cerca del 91%

Esto no es bueno ya que si hubieramos utilizado otro modelo o hubieramos hecho algunos cambios en este, habria aumentado bastante hasta tener una precision de 99.7% por ejemplo.